In [1]:
import requests
import rich
%load_ext rich
from rich import print
from gzip import decompress
import json
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display
import zstandard

In [5]:
r=requests.get("https://yaqwsx.github.io/jlcparts/data/index.json")
categories=r.json()["categories"]
categories


{
    'ADC/DAC/Data Conversion': {
        'ADC/DAC - Specialized': {
            'datahash': '428d337bc172f297efd491698d88f0581b3450d9e7f35f02edecd993830b3d8c',
            'sourcename': 'ADCakaDACakaData_ConversionADCakaDAC___Specialized',
            'stockhash': '19294b06064ac1cf933985c7f9c928d7a17628f4796e415cca676ba72834a12e'
        },
        'Analog Front End (AFEs)': {
            'datahash': '5e7bf435e06ba98f8206f9dfea4b15c889e23332856266a2522a72c4de0dd4e1',
            'sourcename': 'ADCakaDACakaData_ConversionAnalog_Front_End__AFEs_',
            'stockhash': '20081a0d4c19c31e3375933220f51485031d2d5d2f30d13c188f18f175aa56ee'
        },
        'Analog To Digital Converters (ADCs)': {
            'datahash': '708171d1b860d357844d6a7925fa9c2c00bd1b95a5e09c48217352a8acfe7dfe',
            'sourcename': 'ADCakaDACakaData_ConversionAnalog_To_Digital_Converters__ADCs_',
            'stockhash': '691edce18fbf9a04efee9d1729c8f59fd633b4856bd655954987924904070e6d'
        },
      

In [31]:

f = widgets.IntProgress(min=0, max=len(categories)) # instantiate the bar
display(f) # display the bar
inner_bar = widgets.IntProgress(min=0, max=10) # instantiate the bar
display(inner_bar) # display the bar
output=widgets.Output(layout={'border': '1px solid black'})
display(output)
@output.capture()

def fetch_data():
    components=pd.DataFrame()
    global f, inner_bar, output
    for cat, subcats in categories.items():
        inner_bar.max=len(subcats)
        inner_bar.value=0
        inner_bar.description=cat
        for subcat_name, details in subcats.items():
            output.clear_output(wait=True)
            while True:
                try:
                    
                    print(subcat_name)
                    r=requests.get("https://yaqwsx.github.io/jlcparts/data/"+details["sourcename"]+".json.gz",timeout=10)
                    subcat_dict=json.loads(decompress(r.content))
                    subcat=pd.DataFrame(columns=subcat_dict["schema"]+["category","subcategory"])
                    subcat[subcat_dict["schema"]]=np.asarray(subcat_dict["components"],dtype="object")
                    subcat["subcategory"]=subcat_name
                    subcat["category"]=cat
                    
                    r=requests.get("https://yaqwsx.github.io/jlcparts/data/"+details["sourcename"]+".stock.json",timeout=10)
                    catstock=pd.DataFrame(json.loads(r.content).items(),columns=["lcsc","stock"])
                    subcat=pd.merge(subcat,catstock, on="lcsc")
                    if components.empty:
                        components=subcat
                    else:
                        components=pd.concat([components,subcat])
                    inner_bar.value+=1
                except requests.exceptions.Timeout:
                    continue
                break
        f.value+=1
    return components

        
components= fetch_data()
components.to_pickle("cache/components.pickle",compression="zstd")
components
#print(stock)


       

IntProgress(value=0, max=113)

IntProgress(value=0, max=10)

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

,lcsc,mfr,joints,description,datasheet,price,img,url,attributes,category,subcategory,stock
0,C239896,AD421BRZRL7,5,16 SOIC-16-300mil ADCs/DACs - Special Purpose...,https://wmsc.lcsc.com/wmsc/upload/file/pdf/v2/...,"[{'price': 7.49, 'qFrom': 1, 'qTo': 9}, {'pric...",20211009_Analog-Devices-AD421BRZRL7_C239896_fr...,ADC-DAC---Specialized_Analog-Devices-Analog-De...,"{'Basic/Extended': {'format': '${default}', 'p...",ADC/DAC/Data Conversion,ADC/DAC - Specialized,3
1,C459871,AD2S1210WDSTZRL7,48,4.75V~5.25V LQFP-48(7x7) ADCs/DACs - Special ...,https://wmsc.lcsc.com/wmsc/upload/file/pdf/v2/...,"[{'price': 8.491428571, 'qFrom': 1, 'qTo': 9},...",20230106_Analog-Devices-AD2S1210WDSTZRL7_C4598...,ADC-DAC---Specialized_Analog-Devices-Analog-De...,"{'Basic/Extended': {'format': '${default}', 'p...",ADC/DAC/Data Conversion,ADC/DAC - Specialized,126
2,C578328,AD598JRZ,20,SOIC-20 ADC/DAC - Specialized ROHS,https://datasheet.lcsc.com/lcsc/2204121100_Ana...,"[{'price': 67.968571429, 'qFrom': 1, 'qTo': 2}...",20230106_Analog-Devices-AD598JRZ_C578328_back.jpg,ADC-DAC---Specialized_Analog-Devices-Analog-De...,"{'Basic/Extended': {'format': '${default}', 'p...",ADC/DAC/Data Conversion,ADC/DAC - Specialized,41
3,C649048,AD1851RZ-REEL7,2,5V 16 PCM SOIC-16-300mil ADCs/DACs - Special ...,https://wmsc.lcsc.com/wmsc/upload/file/pdf/v2/...,"[{'price': 20.268571429, 'qFrom': 1, 'qTo': 2}...",20230130_Analog-Devices-AD1851RZ-REEL7_C649048...,ADC-DAC---Specialized_Analog-Devices-Analog-De...,"{'Basic/Extended': {'format': '${default}', 'p...",ADC/DAC/Data Conversion,ADC/DAC - Specialized,100
4,C649721,AD5412ACPZ-REEL7,2,±1LSB 2.7V~5.5V 12 Bits LFCSP-40(6x6) Digital...,https://wmsc.lcsc.com/wmsc/upload/file/pdf/v2/...,"[{'price': 12.705714286, 'qFrom': 1, 'qTo': 9}...",None,Digital-To-Analog-Converters-DACs_Analog-Devic...,"{'Basic/Extended': {'format': '${default}', 'p...",ADC/DAC/Data Conversion,ADC/DAC - Specialized,0
...,...,...,...,...,...,...,...,...,...,...,...,...
9,C6461551,61300511121,5,3A Straight 250V 5P -40℃~+105℃ 2.54mm Black Pl...,https://wmsc.lcsc.com/wmsc/upload/file/pdf/v2/...,"[{'price': 0.250571429, 'qFrom': 2, 'qTo': 98}...",None,None,"{'Basic/Extended': {'format': '${default}', 'p...",others,others,1137
10,C6620841,FTSH-105-01-F-DV,10,3.4A Tin Standing paste 10P -55℃~+125℃ 1.27mm ...,https://wmsc.lcsc.com/wmsc/upload/file/pdf/v2/...,"[{'price': 1.355714286, 'qFrom': 1, 'qTo': 9},...",None,None,"{'Basic/Extended': {'format': '${default}', 'p...",others,others,1
11,C6968485,IFX007TAUMA1,8,TO-263-7-1 Gate Drivers ROHS,https://wmsc.lcsc.com/wmsc/upload/file/pdf/v2/...,"[{'price': 2.14, 'qFrom': 1, 'qTo': 9}, {'pric...",None,None,"{'Basic/Extended': {'format': '${default}', 'p...",others,others,61
12,C7125816,TPSM63610RDFR,22,B3QFN-22(6.5x7.5) DC-DC Power Modules ROHS,https://wmsc.lcsc.com/wmsc/upload/file/pdf/v2/...,"[{'price': 18.871428571, 'qFrom': 1, 'qTo': 9}...",None,None,"{'Basic/Extended': {'format': '${default}', 'p...",others,others,15


In [2]:
components=pd.read_pickle("cache/components.pickle",compression="zstd")


In [2]:
a=components["attributes"].map(lambda x:[x["values"] for x in x.values()]) #.list.flatten()

NameError: name 'components' is not defined

In [ ]:
def inner_extract_attr(elem,name):
    if name in elem.keys():
        if "primary" not in elem[name]:
            out=elem[name]["values"][elem[name]["default"]][0]
        else:
            out=elem[name]["values"][elem[name]["primary"]][0]
        if out=="NaN":
            out=float("nan")
        return out
    else: 
        return None


def extract_attribute(components, name):
    attr=components["attributes"].map(lambda y:inner_extract_attr(y,name))
    components[name]=attr
    return components
extract_attribute(components,"Basic/Extended")
for attribute in components[components["Basic/Extended"]!="Extended"]["attributes"].map(lambda x: x.keys()).explode().value_counts().to_dict().keys():
    extract_attribute(components,attribute)
components=components.copy()
display(components)

In [10]:
basic=components[components["Basic/Extended"]!="Extended"] 
for cat in basic["subcategory"].unique():
    print(cat)
    display(basic[basic["subcategory"]==cat].dropna(how='all', axis=1))

ADC/DAC - Specialized

,lcsc,mfr,joints,description,datasheet,price,attributes,category,subcategory,stock,Basic/Extended,Manufacturer,Package,Status
38,C2877015,PCM4201PWR,16,24 TSSOP-16 ADCs/DACs - Special Purpose ROHS,https://wmsc.lcsc.com/wmsc/upload/file/pdf/v2/...,"[{'price': 4.567142857, 'qFrom': 5, 'qTo': 45}...","{'Basic/Extended': {'format': '${default}', 'p...",ADC/DAC/Data Conversion,ADC/DAC - Specialized,3,Preferred,Texas Instruments,TSSOP-16,Discontinued


Analog To Digital Converters (ADCs)

,lcsc,mfr,joints,description,datasheet,price,attributes,category,subcategory,stock,Basic/Extended,Manufacturer,Package,Status
115,C679839,LTC2284IUP#PBF,2,14-bit 2.85V~3.4V 105MHz Parallel QFN-64-EP(9x...,https://wmsc.lcsc.com/wmsc/upload/file/pdf/v2/...,"[{'price': 131.428571429, 'qFrom': 5, 'qTo': 1...","{'Basic/Extended': {'format': '${default}', 'p...",ADC/DAC/Data Conversion,Analog To Digital Converters (ADCs),20,Preferred,Analog Devices,QFN-64-EP(9x9),Discontinued


Digital Potentiometers

...

,lcsc,mfr,joints,description,datasheet,price,img,url,attributes,category,...,Status,Operating temperature,Power dissipation (pd),Collector-emitter breakdown voltage (vceo),Transistor type,Collector current (ic),Transition frequency (ft),"Collector-emitter saturation voltage (vce(sat)@ic,ib)",Collector cut-off current (icbo),"Dc current gain (hfe@ic,vce)"
159,C7420324,2SC1623,3,"50V 200mW 135@1.0mA,6.0V 100mA NPN SOT-23 Bipo...",https://wmsc.lcsc.com/wmsc/upload/file/pdf/v2/...,"[{'price': 0.007985714, 'qFrom': 10, 'qTo': 49...",None,None,"{'Basic/Extended': {'format': '${default}', 'p...",Transistors/Thyristors,...,Discontinued,None,NaN,None,None,None,None,None,None,None
160,C7420355,MMBT4401,3,"40V 300mW 100@150mA,1.0V 600mA NPN SOT-23 Bipo...",https://wmsc.lcsc.com/wmsc/upload/file/pdf/v2/...,"[{'price': 0.009914286, 'qFrom': 10, 'qTo': 49...",20230610_hongjiacheng-MMBT4401_C7420355_front.jpg,Bipolar-BJT_hongjiacheng-hongjiacheng-MMBT4401,"{'Basic/Extended': {'format': '${default}', 'p...",Transistors/Thyristors,...,Active,-55℃~+150℃,0.3,40V,NPN,600mA,250MHz,"400mV@150mA,15mA",100nA,"100@150mA,1.0V"
185,C22388837,MMST3904,3,"40V 200mW 40@0.1mA,1.0V 200mA NPN SOT-323 Bipo...",https://wmsc.lcsc.com/wmsc/upload/file/pdf/v2/...,"[{'price': 0.010542857, 'qFrom': 10, 'qTo': 49...",20240615_hongjiacheng-MMST3904_C22388837_front...,Bipolar-BJT_hongjiacheng-hongjiacheng-MMST3904,"{'Basic/Extended': {'format': '${default}', 'p...",Transistors/Thyristors,...,Active,-55℃~+150℃,0.2,40V,NPN,200mA,300MHz,"0.25V@10mA,1.0mA",60nA,"40@0.1mA,1.0V"
186,C28646266,2SC3356,3,"12V 200mW 250@20mA,10V 100mA NPN SOT-23 Bipola...",https://wmsc.lcsc.com/wmsc/upload/file/pdf/v2/...,"[{'price': 0.013457143, 'qFrom': 10, 'qTo': 19...",None,None,"{'Basic/Extended': {'format': '${default}', 'p...",Transistors/Thyristors,...,Discontinued,None,NaN,None,None,None,None,None,None,None


Darlington Transistors

,lcsc,mfr,joints,description,datasheet,price,attributes,category,subcategory,stock,Basic/Extended,Manufacturer,Package,Status
3,C463680,BSP52T3G,4,"2000@10V,500mA 80V NPN 1A 800mW SOT-223-4 Dar...",https://wmsc.lcsc.com/wmsc/upload/file/pdf/v2/...,"[{'price': 0.227714286, 'qFrom': 2, 'qTo': 98}...","{'Basic/Extended': {'format': '${default}', 'p...",Triode/MOS Tube/Transistor,Darlington Transistors,10,Preferred,onsemi,SOT-223-4,Discontinued


Digital Transistors

,lcsc,mfr,joints,description,datasheet,price,attributes,category,subcategory,stock,Basic/Extended,Manufacturer,Package,Status
23,C672389,DCX144EUQ-7-F,6,"68@5mA,5V 1 NPN, 1 PNP-Pre-Biased 200mW 100mA ...",https://wmsc.lcsc.com/wmsc/upload/file/pdf/v2/...,"[{'price': 0.076714286, 'qFrom': 5, 'qTo': 245...","{'Basic/Extended': {'format': '${default}', 'p...",Triode/MOS Tube/Transistor,Digital Transistors,2900,Preferred,Diodes Incorporated,SOT-363-3,Discontinued


In [ ]:
output=list()
for cat, subcats in categories.items():
    for subcat, details in subcats.items():
        with open('cache/'+details["sourcename"]+".json.gz", 'rb') as f:
            with open('cache/'+details["sourcename"]+".stock.json", 'rb') as stockf:
                
                r=f.read()
                components=json.loads(decompress(r))["components"]
                stock=json.loads(stockf.read())
                basic=[x for x in components if x[8]['Basic/Extended']["values"]["default"][0]!="Extended"]
                for x in basic:
                    elem={
                        "category":subcat,
                        "part_number": x[0],
                        "man_part_number": x[1],
                        "description":x[3],
                        "price": x[5],
                        "basic": x[8]['Basic/Extended']["values"]["default"][0],
                        "manufacturer": x[8]['Manufacturer']["values"]["default"][0],
                        "package": x[8]['Package']["values"]["default"][0],
                        "datasheet": x[4],
                        "stock": stock[x[0]]
                    }
                    output.append(elem)


In [ ]:
df=pd.DataFrame(output)
df[df["part_number"]=="C2869803"]


In [74]:
list(json.loads(r.content).items())

[('C5349995', 0), ('C578360', 15), ('C578504', 2), ('C578505', 482)]